In [1]:
#!pip install chesslab

In [2]:
import numpy as np
from chesslab.utils import load_pkl
from chesslab.training_tf import fitting
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [3]:
example=2
lr = 0.1
epochs=30
batch_size = 128
test_percent=0.1

path = 'D:/database/ccrl/'
name_data='ccrl_states_elo3.pkl'
name_labels='ccrl_results_elo3.pkl'
save_name='./tmp/tf_elo.3.2'

optim = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_1={
    '.':np.array([ 0, 0, 0],dtype=np.float32),
    'p':np.array([ 0, 0, 1],dtype=np.float32),
    'P':np.array([ 0, 0,-1],dtype=np.float32),
    'r':np.array([ 0, 1, 1],dtype=np.float32),
    'R':np.array([ 0,-1,-1],dtype=np.float32),
    'n':np.array([ 1, 0, 0],dtype=np.float32),
    'N':np.array([-1, 0, 0],dtype=np.float32),
    'b':np.array([ 0, 1, 0],dtype=np.float32),
    'B':np.array([ 0,-1, 0],dtype=np.float32),
    'q':np.array([ 1, 0, 1],dtype=np.float32),
    'Q':np.array([-1, 0,-1],dtype=np.float32),
    'k':np.array([ 1, 1, 0],dtype=np.float32),
    'K':np.array([-1,-1, 0],dtype=np.float32)
}

encoding_2={
    '.':np.array([0,0,0,0],dtype=np.float32),
    'p':np.array([1,0,0,0],dtype=np.float32),
    'P':np.array([0,0,0,1],dtype=np.float32),
    'r':np.array([1,0,1,0],dtype=np.float32),
    'R':np.array([0,1,0,1],dtype=np.float32),
    'n':np.array([1,1,0,0],dtype=np.float32),
    'N':np.array([0,0,1,1],dtype=np.float32),
    'b':np.array([0,1,0,0],dtype=np.float32),
    'B':np.array([0,0,1,0],dtype=np.float32),
    'q':np.array([1,0,0,1],dtype=np.float32),
    'Q':np.array([0,1,1,0],dtype=np.float32),
    'k':np.array([1,1,1,0],dtype=np.float32),
    'K':np.array([0,1,1,1],dtype=np.float32)
}


In [6]:
class Model_2():

    def __init__(self,
                   n_classes=2):
        initializer = tf.keras.initializers.GlorotNormal()
        self.hw=[]
        self.hb=[]

        self.hw.append( tf.Variable(initializer(shape=(3,3,4,1,8),dtype=np.float32),name="hl1weigths",dtype="float32") )
        self.hb.append( tf.Variable(np.zeros(8,dtype=np.float32),name="hl1bias",dtype="float32") )
        #8x8x32
        self.hw.append( tf.Variable(initializer(shape=(3,3,4,8,16),dtype=np.float32),name="hl2weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(16,dtype=np.float32),name="hl2bias",dtype="float32"))
        #8x8x64
        self.hw.append( tf.Variable(initializer(shape=(3,3,4,16,32),dtype=np.float32),name="hl3weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(32,dtype=np.float32),name="hl3bias",dtype="float32"))
        #8x8x128
        self.hw.append( tf.Variable(initializer(shape=(8*8*4*32,256),dtype=np.float32),name="hl4weigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(256,dtype=np.float32),name="hl4bias",dtype="float32"))

        self.hw.append( tf.Variable(initializer(shape=(256, n_classes),dtype=np.float32),name="outweigths",dtype="float32"))
        self.hb.append( tf.Variable(np.zeros(n_classes,dtype=np.float32),name="outbias",dtype="float32"))

        self.trainable_variables = []
        for i in range(len(self.hw)):
            self.trainable_variables.append(self.hw[i])    
            self.trainable_variables.append(self.hb[i])

    def __call__(self,x): 

        out = tf.cast(x, tf.float32)
        out = tf.reshape(out, shape=[-1, 8, 8, 4, 1])

        layer=0
        out = tf.nn.conv3d(out,self.hw[layer], strides=[1,1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*32
        layer+=1
        out = tf.nn.conv3d(out,self.hw[layer], strides=[1,1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)
              #8*8*64
        layer+=1  
        out = tf.nn.conv3d(out,self.hw[layer], strides=[1,1,1,1,1], padding='SAME')  
        out = tf.add(out, self.hb[layer])
        out = tf.nn.elu(out)
              #8*8*128
        layer+=1
        out =  tf.reshape(out,[-1, 8*8*4*32])
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])
        out = tf.nn.relu(out)

        layer+=1
        out =  tf.matmul(out,self.hw[layer])
        out = tf.add(out, self.hb[layer])

        return out

In [7]:
np.random.seed(0)
tf.random.set_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: pasa de onehot a logits

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(2661290, 64)
(2661290,)


In [8]:
if example==1:
    model = Model_1()
    encoding=encoding_1
else:
    model = Model_2()
    encoding=encoding_2


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model=model,
        optimizer=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding)

2021-10-29 19:05:07


Epoch: 01/30 | time: 158s = 2.6m | train loss: 0.5352 | train acc: 0.7240 | test loss: 0.4765 | test acc: 0.7673

Epoch: 02/30 | time: 154s = 2.6m | train loss: 0.3771 | train acc: 0.8249 | test loss: 0.3454 | test acc: 0.8453

Epoch: 03/30 | time: 155s = 2.6m | train loss: 0.2518 | train acc: 0.8897 | test loss: 0.2471 | test acc: 0.8940

Epoch: 04/30 | time: 156s = 2.6m | train loss: 0.1805 | train acc: 0.9231 | test loss: 0.2480 | test acc: 0.9034

...

Epoch: 26/30 | time: 155s = 2.6m | train loss: 0.0368 | train acc: 0.9869 | test loss: 0.1829 | test acc: 0.9567

Epoch: 27/30 | time: 155s = 2.6m | train loss: 0.0357 | train acc: 0.9873 | test loss: 0.4456 | test acc: 0.9247

Epoch: 28/30 | time: 155s = 2.6m | train loss: 0.0361 | train acc: 0.9873 | test loss: 0.1906 | test acc: 0.9572

Epoch: 29/30 | time: 155s = 2.6m | train loss: 0.0366 | train acc: 0.9872 | test loss: 0.1781 | test acc: 0.9589

Epoch: 30/30 | time: 155s = 2.6m | train loss: 0.0358 | train acc: 0.9875 | test loss: 0.1961 | test acc: 0.9566

fitting(epochs=1,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        model= model, 
        load_name='tmp/tf_weights.0.6.h5',
        save_name=save_name,)